In [1]:
!pip install selenium

In [ ]:
from selenium import webdriver

chromedriver_path = r"D:\Programming\Bootcamp#27\Class7\chromedriver.exe"

driver = webdriver.Chrome(executable_path=chromedriver_path)
driver

In [ ]:
!pip install webdriver-manager

In [ ]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

driver = webdriver.Chrome(service=Service(chromedriver_path))

In [ ]:
driver.get('https://shopee.co.th/')

In [ ]:
from selenium.webdriver.common.by import By

In [ ]:
thai_button_element = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
thai_button_element

In [ ]:
button_element = driver.find_element(By.CLASS_NAME, 'shopee-button-outline')
button_element

In [ ]:
thai_button_element.click()

driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/shopee-banner-popup-stateful//div/div/div/div/div/svg')

In [ ]:
close_popup_element = driver.execute_script("return document.querySelector('shopee-banner-popup-stateful').shadowRoot.querySelector('div.shopee-popup__close-btn')")
close_popup_element

In [ ]:
close_popup_element.click()

In [ ]:
search_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/header/div[2]/div/div[1]/div[1]/div/form/input')
search_element.send_keys('เก้าอี้ ergonomic')

In [ ]:
from selenium.webdriver.common.keys import Keys

In [ ]:
search_element.send_keys(Keys.ENTER)

In [ ]:
driver.execute_script("document.body.style.zoom='10%'")

In [ ]:
data = driver.page_source
data

In [ ]:
import bs4
soup = bs4.BeautifulSoup(data)
soup

In [ ]:
all_product = soup.find_all('div',{'class':"col-xs-2-4 shopee-search-item-result__item"})
all_product

In [ ]:
product = all_product[0]
title_element = product.find('div',{'class':'ie3A+n bM+7UW Cve6sh'})
title_text = title_element.text
title_text

price_element = product.find('div',{'class':'hpDKMN'})
price_text = price_element.text
price_text

sales_element = product.find('div',{'class':'r6HknA uEPGHT'})
if sales_element is None:
    sales_text = None
else:
    sales_text = sales_element.text

product_link_element = product.find('a')
product_link = product_link_element.get('href')
product_link

In [ ]:
data_dict = dict()
data_dict['title'] = title_text
data_dict['price'] = price_text
data_dict['sales'] = sales_text
data_dict['link'] = product_link
data_dict


In [ ]:
data_dict_list = []
for product in all_product:
    title_element = product.find('div',{'class':'ie3A+n bM+7UW Cve6sh'})
    title_text = title_element.text

    price_element = product.find('div',{'class':'hpDKMN'})
    price_text = price_element.text

    sales_element = product.find('div',{'class':'r6HknA uEPGHT'})
    if sales_element is None:
        sales_text = None
    else:
        sales_text = sales_element.text

    product_link_element = product.find('a')
    product_link = product_link_element.get('href')
    
    data_dict = dict()
    data_dict['title'] = title_text
    data_dict['price'] = price_text
    data_dict['sales'] = sales_text
    data_dict['link'] = product_link
    data_dict_list.append(data_dict)


In [ ]:
data_dict_list

In [ ]:
import pandas as pd
data_df = pd.DataFrame(data_dict_list)
data_df

In [ ]:
price_1 = '฿6,698฿4,699'
price_2 = '฿2,616 - ฿2,772'
price_3 = '฿961'

In [ ]:
import re

In [ ]:
price = price_2
def clean_price(price):
    result = re.findall(r'[0-9.,]+',price)
    if '-' in price:
        original_price = None
        min_price = result[0]
        max_price = result[1]
    elif len(result) == 2:
        original_price = result[0]
        min_price = result[1]
        max_price = result[1]
    elif len(result) == 1:
        original_price = result[0]
        min_price = result[0]
        max_price = result[0]
        
    price_list =  [original_price,min_price,max_price]
    price_clean_list = []
    for price in price_list:
        if price is None:
            price_clean_list.append(None)
        else:
            price_clean_list.append(int(price.replace(',','')))
    
    return price_clean_list


In [ ]:
price_clean_ser = data_df['price'].apply(clean_price)
price_clean_df = pd.DataFrame(price_clean_ser)
price_clean_df

In [ ]:
price_clean_list = list(price_clean_ser)
price_clean_df = pd.DataFrame(price_clean_list, columns=['Original Price', 'Min Price', 'Max Price'])
price_clean_df

In [ ]:
price_data_df = pd.concat([data_df,price_clean_df],axis=1)
price_data_df

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
import pandas as pd
import bs4
import re

In [ ]:
def get_data():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get('https://shopee.co.th/')
    
    thai_button = '/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button'
    thai_button_element = driver.find_element(By.XPATH, thai_button)
    thai_button_element.click() 
    close_popup_element = driver.execute_script("return document.querySelector('shopee-banner-popup-stateful').shadowRoot.querySelector('div.shopee-popup__close-btn')")
    close_popup_element.click() 
    # Click to get rid of popups
    
    search_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/header/div[2]/div/div[1]/div[1]/div/form/input')
    search_element.send_keys('เก้าอี้ ergonomic')
    search_element.send_keys(Keys.ENTER)
    
    driver.execute_script("document.body.style.zoom='10%'")
    time.sleep(5)
    data = driver.page_source
    # Wait 5 second for whole page to load before getting page source
    
    return data

def parse_data(data):
    soup = bs4.BeautifulSoup(data)
    all_product = soup.find_all('div',{'class':'col-xs-2-4 shopee-search-item-result__item'}) 
    data_dict_list = []
    for product in all_product:
        title_element = product.find('div',{'class':'ie3A+n bM+7UW Cve6sh'})
        title_text = title_element.text

        price_element = product.find('div',{'class':'hpDKMN'})
        price_text = price_element.text

        sales_element = product.find('div',{'class':'r6HknA uEPGHT'})
        if sales_element is None:
            sales_text = None
        else:
            sales_text = sales_element.text
               
        product_link_element = product.find('a')
        product_link = product_link_element.get('href')
        
        data_dict = dict()
        data_dict['title'] = title_text
        data_dict['price'] = price_text
        data_dict['sales'] = sales_text
        data_dict['link'] = product_link
        data_dict_list.append(data_dict)
    return data_dict_list

def clean_price(price):
    result = re.findall(r'[0-9.,]+',price)
    if '-' in price:
        original_price = None
        min_price = result[0]
        max_price = result[1]
    elif len(result) == 2:
        original_price = result[0]
        min_price = result[1]
        max_price = result[1]
    elif len(result) == 1:
        original_price = result[0]
        min_price = result[0]
        max_price = result[0]
        
    price_list =  [original_price,min_price,max_price]
    price_clean_list = []
    for price in price_list:
        if price is None:
            price_clean_list.append(None)
        else:
            price_clean_list.append(int(price.replace(',','')))
    
    return price_clean_list

def clean_data(data_dict_list):
    data_df = pd.DataFrame(data_dict_list)
    price_clean_ser = data_df['price'].apply(clean_price)
    price_clean_list = list(price_clean_ser)
    price_clean_df = pd.DataFrame(price_clean_list, columns=['Original Price', 'Min Price', 'Max Price'])
    price_data_df = pd.concat([data_df,price_clean_df],axis=1)
    price_data_df = price_data_df.drop(columns=['price'])
    
    return price_data_df

In [ ]:
data = get_data()
data_dict_list = parse_data(data)
df = clean_data(data_dict_list)
df